## Documentation

To read more about the delete API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-delete.html).

![delete_documents_docs](../images/delete_documents_docs.png)

## Connect to ElasticSearch

In [4]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'es-docker-cluster',
 'cluster_uuid': '68vsKryIR7Ss49bLh7mz5Q',
 'name': 'es01',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


## Index documents

In [5]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `document_ids` list.

In [6]:
import json
from tqdm import tqdm


document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='my_index', body=document)
    document_ids.append(response['_id'])

100%|██████████| 3/3 [00:00<00:00, 30.89it/s]


In [7]:
document_ids

['_z0Ya5sBX61qEH3bHUio', 'AD0Ya5sBX61qEH3bHUn-', 'AT0Ya5sBX61qEH3bHkkE']

## Delete operation

This is an example of a successful operation. If the `ID` exists in the document, the `delete` operation won't return any errors.

In [8]:
response = es.delete(index='my_index', id=document_ids[0])

In [ ]:
#!/bin/sh
curl -X DELETE "http://localhost:9200/my_index/_doc/_z0Ya5sBX61qEH3bHUio"  # .../_id

# Kibana Dev Tools equivalent:
DELETE /my_index/_doc/_z0Ya5sBX61qEH3bHUio   # -- GET /my_index/_doc/_id

In [9]:
from pprint import pprint

pprint(response.body)

{'_id': '_z0Ya5sBX61qEH3bHUio',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'deleted'}


This example shows that the `delete` operation fails when providing it with an `ID` that does not exist in the index.

In [10]:
try:
    response = es.delete(index='my_index', id="id")
except Exception as e:
    print(e)

NotFoundError(404, "{'_index': 'my_index', '_id': 'id', '_version': 1, 'result': 'not_found', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}")
